# Install SelfAttention

In [ ]:
#pip install keras-self-attention

# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Conv1D, MaxPooling1D, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint
#from tensorflow_addons.layers import SeqSelfAttention
from keras_self_attention import SeqSelfAttention
from keras.utils import to_categorical
import numpy as np

In [ ]:
X = np.load(r'/content/drive/My Drive/Model Files/X_train.npy')
y = np.load(r'/content/drive/My Drive/Model Files/y_train.npy')

In [ ]:
num_classes = 94
y_one_hot = np.zeros((y.shape[0], y.shape[1], num_classes), dtype=np.int32)
for i in range(y.shape[0]):
    y_one_hot[i] = to_categorical(y[i], num_classes=num_classes)

# Model Architectures

In [ ]:
epochs = 50
batch_size = 32

## BiLSTM + SelfAttention

In [ ]:
# Creating Bidirectional LSTM model (with Attention Layer)

max_sequence_length = 5
feature_vector_size = len(X[0][0])
model = Sequential()
model.add(SeqSelfAttention(attention_activation='sigmoid', input_shape=(max_sequence_length, feature_vector_size)))
model.add(LSTM(units=2048, return_sequences=True))
model.add(TimeDistributed(Dense(units=num_classes, activation='softmax')))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## LSTM with 1DConvolutional

In [ ]:
# Creating LSTM model (with 1D Convolution Layer)

max_sequence_length = 5
feature_vector_size = len(X[0][0])
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(max_sequence_length, feature_vector_size)))
model.add(LSTM(units=2048, return_sequences=True))
model.add(TimeDistributed(Dense(units=num_classes, activation='softmax')))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Multiple LSTMs with Dropout

In [ ]:
# Creating Multiple LSTMs model with dropout

max_sequence_length = 5
feature_vector_size = len(X[0][0])
model = Sequential()
model.add(LSTM(units=2048, input_shape=(max_sequence_length, feature_vector_size), return_sequences = True)) # sequence-to-sequence
model.add(LSTM(units=1024, return_sequences = True))
model.add(LSTM(units=512, return_sequences = True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(units=num_classes, activation='softmax')))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model Training

In [ ]:
model.fit(X, y_one_hot, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/50
183/183 [==============================] - 7s 22ms/step - loss: 3.1776 - accuracy: 0.1931 - val_loss: 3.2383 - val_accuracy: 0.1630
Epoch 2/50
183/183 [==============================] - 3s 17ms/step - loss: 1.8433 - accuracy: 0.4352 - val_loss: 3.4489 - val_accuracy: 0.2172
Epoch 3/50
183/183 [==============================] - 3s 17ms/step - loss: 1.3096 - accuracy: 0.5814 - val_loss: 3.7351 - val_accuracy: 0.2374
Epoch 4/50
183/183 [==============================] - 3s 17ms/step - loss: 0.9821 - accuracy: 0.6851 - val_loss: 3.4607 - val_accuracy: 0.2669
Epoch 5/50
183/183 [==============================] - 3s 17ms/step - loss: 0.7696 - accuracy: 0.7526 - val_loss: 4.0018 - val_accuracy: 0.2376
Epoch 6/50
183/183 [==============================] - 3s 17ms/step - loss: 0.5920 - accuracy: 0.8075 - val_loss: 4.1326 - val_accuracy: 0.2823
Epoch 7/50
183/183 [==============================] - 3s 17ms/step - loss: 0.4775 - accuracy: 0.8449 - val_loss: 4.0598 - val_accuracy: 0.2804

In [ ]:
model.save(r'/content/drive/My Drive/Model Files/Separated_Convolutional.h5')

# Train and Test Accuracies

In [ ]:
train_results = model.evaluate(X, y_one_hot)
print("Train Loss:", train_results[0])
print("Train Accuracy:", train_results[1])

229/229 [==============================] - 2s 7ms/step - loss: 1.2244 - accuracy: 0.8593
Train Loss: 1.224387764930725
Train Accuracy: 0.8593343496322632


In [ ]:
X_test = np.load(r'/content/drive/My Drive/Model Files/X_test.npy')
y_test = np.load(r'/content/drive/My Drive/Model Files/y_test.npy')

In [ ]:
y_test_ohe = np.zeros((y_test.shape[0], y_test.shape[1], num_classes), dtype=np.int32)
for i in range(y_test.shape[0]):
    y_test_ohe[i] = to_categorical(y_test[i], num_classes=num_classes)

In [ ]:
test_results = model.evaluate(X_test, y_test_ohe)
print("Test Loss:", test_results[0])
print("Test Accuracy:", test_results[1])

58/58 [==============================] - 0s 7ms/step - loss: 1.5984 - accuracy: 0.7768
Test Loss: 1.598362922668457
Test Accuracy: 0.7767643928527832
